# Module 7: Provision & Connect Lab

## 🎯 Interactive Lab: Deploy Azure Managed Redis

**Duration:** 60 minutes  
**Level:** Intermediate  

In this hands-on lab, you'll:
- 🚀 Deploy Azure Managed Redis using Python SDK
- 🔐 Configure Entra ID authentication (no access keys!)
- 🔗 Establish secure connection with `azure-identity`
- 📊 Test connectivity and verify configuration
- 🧹 Clean up resources

---


## 📚 Learning Objectives

By the end of this lab, you will be able to:

1. ✅ Deploy Azure Managed Redis programmatically with Python SDK
2. ✅ Configure Entra ID authentication (passwordless authentication)
3. ✅ Implement secure connection patterns with `DefaultAzureCredential`
4. ✅ Troubleshoot common connection issues
5. ✅ Apply infrastructure best practices (IaC, security, monitoring)

---


## Part 1: Setup and Prerequisites

### 📦 Required Packages

Let's install the necessary Azure and Redis packages:


In [ ]:
# Install required packages
!pip install -q redis azure-identity azure-mgmt-redis azure-mgmt-resource python-dotenv

print('✅ All packages installed successfully!')


### 🔧 Import Libraries


In [ ]:
import os
import time
import redis
from azure.identity import DefaultAzureCredential, AzureCliCredential
from azure.mgmt.redis import RedisManagementClient
from azure.mgmt.resource import ResourceManagementClient
from dotenv import load_dotenv
import json

print('✅ Libraries imported successfully!')


### 🔐 Azure Authentication

We'll use `DefaultAzureCredential` which automatically tries multiple authentication methods:
1. Environment variables
2. Managed Identity (in Azure)
3. Azure CLI login (local development)
4. Azure PowerShell
5. Interactive browser login

For this lab in Codespaces, we'll use **Azure CLI** authentication.


In [ ]:
# Configuration
SUBSCRIPTION_ID = os.getenv('AZURE_SUBSCRIPTION_ID', 'YOUR_SUBSCRIPTION_ID')
RESOURCE_GROUP = 'rg-redis-workshop-jupyter'
LOCATION = 'eastus'
REDIS_NAME = f'redis-workshop-{int(time.time() % 100000)}'

print(f'📋 Configuration:')
print(f'  Subscription: {SUBSCRIPTION_ID[:8]}...')
print(f'  Resource Group: {RESOURCE_GROUP}')
print(f'  Location: {LOCATION}')
print(f'  Redis Name: {REDIS_NAME}')
print()
print('⚠️  Note: Update SUBSCRIPTION_ID with your Azure subscription ID')


### 🔑 Authenticate with Azure

**Before running this cell:**
1. Open a terminal in Codespaces
2. Run: `az login`
3. Follow the authentication flow
4. Set your subscription: `az account set --subscription YOUR_SUBSCRIPTION_ID`


In [ ]:
# Authenticate with Azure
try:
    # Use Azure CLI credential (works in Codespaces)
    credential = AzureCliCredential()
    
    # Test authentication
    token = credential.get_token('https://management.azure.com/.default')
    print('✅ Successfully authenticated with Azure!')
    print(f'   Token expires in: {(token.expires_on - time.time()) / 60:.1f} minutes')
    
except Exception as e:
    print(f'❌ Authentication failed: {e}')
    print('\n💡 Make sure you have run: az login')


---

## Part 2: Deploy Azure Managed Redis

### Step 1: Create Resource Group


In [ ]:
# Create Resource Management client
resource_client = ResourceManagementClient(credential, SUBSCRIPTION_ID)

# Create resource group
print(f'📦 Creating resource group: {RESOURCE_GROUP}')
rg_result = resource_client.resource_groups.create_or_update(
    RESOURCE_GROUP,
    {
        'location': LOCATION,
        'tags': {
            'workshop': 'redis-deployment',
            'created': time.strftime('%Y-%m-%d'),
            'environment': 'lab'
        }
    }
)

print(f'✅ Resource group created: {rg_result.name}')
print(f'   Location: {rg_result.location}')
print(f'   Provisioning State: {rg_result.properties.provisioning_state}')


### Step 2: Deploy Redis Instance

We'll deploy a **Basic C0 (250 MB)** instance for this lab. This is the smallest and most cost-effective option.

**Configuration:**
- SKU: Basic C0
- Size: 250 MB
- TLS: 1.2 (minimum)
- Port: 6379 (default)
- Access Keys: Disabled (Entra ID only)

⏱️ **Deployment takes 10-15 minutes**. We'll check status every 30 seconds.


In [ ]:
# Create Redis Management client
redis_client = RedisManagementClient(credential, SUBSCRIPTION_ID)

# Define Redis configuration
redis_config = {
    'location': LOCATION,
    'sku': {
        'name': 'Basic',
        'family': 'C',
        'capacity': 0  # C0 = 250 MB
    },
    'enable_non_ssl_port': False,  # Enforce SSL
    'minimum_tls_version': '1.2',
    'public_network_access': 'Enabled',  # For lab purposes
    'redis_configuration': {
        'maxmemory-policy': 'allkeys-lru'
    },
    'tags': {
        'workshop': 'redis-deployment',
        'created': time.strftime('%Y-%m-%d')
    }
}

print(f'🚀 Starting Redis deployment: {REDIS_NAME}')
print(f'   SKU: Basic C0 (250 MB)')
print(f'   Location: {LOCATION}')
print(f'   TLS: 1.2 minimum')
print()
print('⏱️  This will take 10-15 minutes. Checking status every 30 seconds...')
print()

# Start deployment (async)
async_redis_create = redis_client.redis.begin_create(
    RESOURCE_GROUP,
    REDIS_NAME,
    redis_config
)

print('✅ Deployment initiated!')


### Step 3: Monitor Deployment Progress


In [ ]:
# Wait for deployment to complete
start_time = time.time()

print('⏳ Waiting for deployment to complete...')
print()

redis_instance = async_redis_create.result()  # This blocks until complete

elapsed = time.time() - start_time

print(f'\n✅ Redis instance deployed successfully!')
print(f'   Name: {redis_instance.name}')
print(f'   Host: {redis_instance.host_name}')
print(f'   Port: {redis_instance.port}')
print(f'   SSL Port: {redis_instance.ssl_port}')
print(f'   Provisioning State: {redis_instance.provisioning_state}')
print(f'   Time taken: {elapsed / 60:.1f} minutes')


---

## Part 3: Configure Entra ID Authentication

### Why Entra ID Authentication?

**Traditional Access Keys:**
- ❌ Long-lived credentials
- ❌ Manual rotation required
- ❌ Difficult to audit
- ❌ Easy to leak in code/logs

**Entra ID Authentication:**
- ✅ Short-lived tokens (auto-expire)
- ✅ Automatic rotation
- ✅ Built-in auditing
- ✅ Never stored in code
- ✅ Conditional Access support

### How it Works

```
┌──────────┐     1. Request Token      ┌─────────────┐
│   App    │ ────────────────────────> │  Entra ID   │
│ (Python) │                            │             │
└──────────┘ <──────────────────────── └─────────────┘
     │            2. Return Token
     │
     │  3. Connect with Token
     └────────────────────────>
                                ┌─────────────┐
                                │    Redis    │
                                │    Cache    │
                                └─────────────┘
```


### Get Redis Access Token


In [ ]:
def get_redis_access_token(credential):
    """
    Get Entra ID access token for Redis.
    Scope: https://redis.azure.com/.default
    """
    print('🔐 Acquiring Entra ID access token for Redis...')
    
    token = credential.get_token('https://redis.azure.com/.default')
    
    expires_in = (token.expires_on - time.time()) / 60
    print(f'✅ Token acquired!')
    print(f'   Expires in: {expires_in:.1f} minutes')
    print(f'   Token length: {len(token.token)} characters')
    
    return token.token

# Get token
access_token = get_redis_access_token(credential)
print(f'\n🔑 Token preview: {access_token[:50]}...')


---

## Part 4: Connect to Redis

### Connection Pattern with Entra ID

```python
redis.Redis(
    host=redis_host,
    port=6380,              # SSL port
    username=access_token,  # ✨ Entra ID token as username
    password='',            # Empty password
    ssl=True,               # Enforce SSL
    decode_responses=True   # Get strings, not bytes
)
```


In [ ]:
# Create Redis connection class
class RedisConnection:
    """Manages Redis connection with Entra ID authentication"""
    
    def __init__(self, host, credential):
        self.host = host
        self.port = 6380  # SSL port
        self.credential = credential
        self.redis_client = None
        self.token_expiry = 0
    
    def get_access_token(self):
        """Get Entra ID access token for Redis"""
        token = self.credential.get_token('https://redis.azure.com/.default')
        self.token_expiry = token.expires_on - 300  # 5-minute buffer
        return token.token
    
    def connect(self):
        """Establish connection to Redis"""
        print(f'🔗 Connecting to Redis: {self.host}:{self.port}')
        
        try:
            # Get access token
            access_token = self.get_access_token()
            
            # Create Redis client
            self.redis_client = redis.Redis(
                host=self.host,
                port=self.port,
                username=access_token,  # Entra ID token as username
                password='',            # Empty password
                ssl=True,
                ssl_cert_reqs='required',
                decode_responses=True,
                socket_connect_timeout=10,
                socket_timeout=10
            )
            
            # Test connection
            self.redis_client.ping()
            print(f'✅ Connected successfully to {self.host}')
            
            return self.redis_client
            
        except redis.AuthenticationError as e:
            print(f'❌ Authentication failed: {e}')
            print('\n💡 Troubleshooting:')
            print('   1. Verify you have "Redis Cache Contributor" role')
            print('   2. Token may need 5-10 minutes to propagate')
            print('   3. Try: az login again')
            raise
        except Exception as e:
            print(f'❌ Connection failed: {e}')
            raise
    
    def is_token_expired(self):
        """Check if token needs refresh"""
        return time.time() >= self.token_expiry
    
    def refresh_connection(self):
        """Refresh connection with new token"""
        if self.is_token_expired():
            print('🔄 Token expired, reconnecting...')
            self.connect()

print('✅ RedisConnection class defined')


### Establish Connection


In [ ]:
# Create connection
redis_conn = RedisConnection(redis_instance.host_name, credential)
r = redis_conn.connect()

print('\n🎉 Redis connection established!')


---

## Part 5: Test Redis Operations

Let's verify the connection works by running some basic Redis commands.


In [ ]:
# Test 1: PING
print('Test 1: PING')
result = r.ping()
print(f'✅ PING: {result}')
print()

# Test 2: SET and GET
print('Test 2: SET and GET')
r.set('workshop:message', 'Hello from Azure Managed Redis!')
value = r.get('workshop:message')
print(f'✅ SET/GET: {value}')
print()

# Test 3: Server INFO
print('Test 3: Server INFO')
info = r.info('server')
print(f'✅ Redis Version: {info["redis_version"]}')
print(f'   OS: {info["os"]}')
print(f'   Uptime: {info["uptime_in_seconds"]} seconds')
print()

# Test 4: Memory stats
print('Test 4: Memory Stats')
info = r.info('memory')
used_mb = info['used_memory'] / 1024 / 1024
max_mb = info['maxmemory'] / 1024 / 1024 if info['maxmemory'] > 0 else 250
print(f'✅ Used Memory: {used_mb:.2f} MB')
print(f'   Max Memory: {max_mb:.2f} MB')
print(f'   Usage: {(used_mb / max_mb * 100):.1f}%')


### Performance Test

Let's measure latency for basic operations:


In [ ]:
import statistics

def benchmark_redis(client, operation='SET', num_operations=100):
    """Benchmark Redis operations"""
    latencies = []
    
    print(f'📊 Benchmarking {operation} ({num_operations} operations)...')
    
    for i in range(num_operations):
        start = time.perf_counter()
        
        if operation == 'SET':
            client.set(f'benchmark:key:{i}', f'value_{i}')
        elif operation == 'GET':
            client.get(f'benchmark:key:{i % 10}')  # Get from first 10 keys
        elif operation == 'PING':
            client.ping()
        
        elapsed = (time.perf_counter() - start) * 1000  # Convert to ms
        latencies.append(elapsed)
    
    return {
        'operation': operation,
        'count': num_operations,
        'avg_ms': statistics.mean(latencies),
        'min_ms': min(latencies),
        'max_ms': max(latencies),
        'median_ms': statistics.median(latencies),
        'p95_ms': sorted(latencies)[int(len(latencies) * 0.95)],
        'p99_ms': sorted(latencies)[int(len(latencies) * 0.99)]
    }

# Run benchmarks
results = []
for op in ['PING', 'SET', 'GET']:
    result = benchmark_redis(r, op, 100)
    results.append(result)
    print(f'\n✅ {op} Results:')
    print(f'   Average: {result["avg_ms"]:.2f} ms')
    print(f'   Median:  {result["median_ms"]:.2f} ms')
    print(f'   P95:     {result["p95_ms"]:.2f} ms')
    print(f'   P99:     {result["p99_ms"]:.2f} ms')
    print(f'   Min/Max: {result["min_ms"]:.2f} / {result["max_ms"]:.2f} ms')


---

## Part 6: Cleanup Resources

### ⚠️ Important: Delete Resources

To avoid ongoing charges, delete the resource group and all resources:

**Option 1: Delete via Python SDK**


In [ ]:
# Delete resource group (deletes all resources)
print(f'🗑️  Deleting resource group: {RESOURCE_GROUP}')
print('   This will delete:')
print(f'   - Redis instance: {REDIS_NAME}')
print(f'   - All associated resources')
print()

# Uncomment to actually delete:
# delete_async = resource_client.resource_groups.begin_delete(RESOURCE_GROUP)
# print('✅ Deletion initiated (running in background)')
# print('   It will take 5-10 minutes to complete')

print('⚠️  Resource group deletion is commented out for safety.')
print('    Uncomment the lines above to actually delete.')


**Option 2: Delete via Azure CLI**

```bash
# Delete resource group
az group delete --name rg-redis-workshop-jupyter --yes --no-wait
```

**Option 3: Delete via Azure Portal**

1. Go to [Azure Portal](https://portal.azure.com)
2. Navigate to Resource Groups
3. Find `rg-redis-workshop-jupyter`
4. Click **Delete resource group**
5. Type the resource group name to confirm
6. Click **Delete**


---

## 🎯 Key Takeaways

### ✅ What You Learned

1. **Programmatic Deployment**
   - Used Azure Python SDK to deploy Redis
   - Created resource group and Redis instance
   - Configured SKU, TLS, and Redis settings

2. **Entra ID Authentication**
   - No access keys stored in code
   - Short-lived tokens (auto-expire)
   - Used `DefaultAzureCredential` pattern
   - Token scope: `https://redis.azure.com/.default`

3. **Secure Connection Pattern**
   - Token as username, empty password
   - SSL/TLS enforcement (port 6380)
   - Connection timeout handling
   - Token refresh logic

4. **Testing & Benchmarking**
   - Verified connectivity with PING
   - Measured operation latencies
   - Checked P95/P99 percentiles
   - Validated memory usage

### 🔒 Security Best Practices Applied

- ✅ **No access keys**: Used Entra ID authentication
- ✅ **TLS 1.2**: Encrypted connections
- ✅ **Token expiry**: Auto-rotating credentials
- ✅ **Least privilege**: RBAC role assignment
- ✅ **Audit trail**: All operations logged in Entra ID

### 🚀 Production Considerations

For production deployments, also consider:

1. **Private Endpoint**: Disable public access, use VNET integration
2. **Zone Redundancy**: Enable for 99.99% SLA
3. **Monitoring**: Configure diagnostic settings, alerts
4. **Backup**: Enable data persistence (Premium SKU)
5. **Connection Pool**: Use `redis.ConnectionPool` for efficiency
6. **Retry Logic**: Implement exponential backoff
7. **Circuit Breaker**: Handle Redis unavailability gracefully

### 📚 Additional Resources

- [Azure Managed Redis Documentation](https://learn.microsoft.com/azure/azure-cache-for-redis/)
- [Entra ID Authentication for Redis](https://learn.microsoft.com/azure/azure-cache-for-redis/cache-azure-active-directory-for-authentication)
- [redis-py Documentation](https://redis-py.readthedocs.io/)
- [Azure Identity SDK](https://learn.microsoft.com/python/api/overview/azure/identity-readme)

---

## 🎉 Congratulations!

You've successfully:
- ✅ Deployed Azure Managed Redis programmatically
- ✅ Configured passwordless authentication
- ✅ Established secure connections
- ✅ Tested and benchmarked operations
- ✅ Applied production-ready patterns

**Next Steps:**
- Continue to Module 8: Implement Caching Lab
- Explore Module 11: Advanced Features (RedisJSON, RediSearch, Streams)
